In [7]:
# hàm dùng chung 

import pandas as pd
import math
import random
import warnings

class dataLoader:
    def __init__(self, data_url, label):
        warnings.filterwarnings('ignore')
        self.lastest_row = "ahihi"
        pd.options.mode.chained_assignment = None
        if data_url.endswith('.csv'):
            self.data = pd.read_csv(data_url)
            self.lastest_row = self.data.iloc[-1]
            self.data = self.data.iloc[:len(self.data)-1]
            print(f"cai gi :{self.lastest_row}")
            
        elif data_url.endswith('.xlsx'):
            self.data = pd.read_excel(data_url)
        else:
            raise ValueError("Unsupported file format. Please provide a CSV or XLSX file.")
        
        self.label = label
        
        if self.label not in self.data.columns.tolist():
            raise Exception("label not in dataset")
        
    def detachedData(self, data):

        Y = data[self.label]
        X = data.drop(columns=[self.label])
        return X,Y

    def hold_out(self, train_per = 0.7, val_per = 0.3):        
        m = len(self.data)
        
        self.train_data = self.data.iloc[0: math.floor(train_per * m)] # 0-105 => 106 pt
            
        self.val_data = self.data.iloc[len(self.train_data) :]
        
        self.train_data.X, self.train_data.Y = self.detachedData(self.train_data)
        
        self.val_data.X, self.val_data.Y = self.detachedData(self.val_data)
        
    def kfold_split(self, K, shuffle=True, random_state=42):
        if shuffle:            
            indices = list(self.data.index)
            random.seed(random_state)
            random.shuffle(indices)
            self.data = self.data.iloc[indices].reset_index(drop=True)
            # chưa hiểu cách chạy self.data.iloc[indices].reset_index(drop=True)

        fold_sizes = np.full(K, len(self.data) // K, dtype=int)
        fold_sizes[: len(self.data) % K] += 1

        current = 0
        folds = []
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            folds.append(self.data.iloc[start:stop])
            current = stop

        return folds
    
    def K_fold(self, K = 5):
        folds = self.kfold_split(K, shuffle=True)  # Tách từng fold cho K

        self.val_data = []
        self.train_data = []

        for i in range(K):
            self.val_data.append(folds[i])  # Chọn fold[i] làm tập validation

            train_j = pd.DataFrame()  # Khởi tạo lại tập huấn luyện trong mỗi vòng lặp
            for j in range(K):
                if i != j:
                    train_j = pd.concat([train_j, folds[j]], axis=0, ignore_index=True)
            self.train_data.append(train_j)  # Thêm train_j vào self.train_data

        for i in range(len(self.train_data)):
            self.train_data[i].X, self.train_data[i].Y = self.detachedData(self.train_data[i])
            
            self.val_data[i].X, self.val_data[i].Y = self.detachedData(self.val_data[i])
            
    def LOO(self):
        self.K_fold(len(self.data))
        
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class Evaluate:
    @staticmethod
    def loss(y_true, y_pred):
        return sum(abs(y_true - y_pred) / len(y_true))
    
    @staticmethod
    def confusion_matrix(y_true, y_pred):
        unique_labels = np.unique(y_true)
        matrix = {label: {l: 0 for l in unique_labels} for label in unique_labels}

        for true, pred in zip(y_true, y_pred):
            matrix[true][pred] += 1

        # Convert to DataFrame
        matrix_df = pd.DataFrame(matrix).T.fillna(0)
        
        return matrix_df

    @staticmethod
    def classification_report(y_true, y_pred):
        cm = Evaluate.confusion_matrix(y_true, y_pred)
        report = {}
        
        for label in cm.index:
            TP = cm.loc[label, label]
            FP = cm[label].sum() - TP
            FN = cm.loc[label].sum() - TP
            TN = cm.sum().sum() - (TP + FP + FN)
            
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            report[label] = {
                'precision': precision,
                'recall': recall,
                'f1_score': f1_score
            }

        # Plot the heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=cm.columns, yticklabels=cm.index)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()

        return report

In [8]:
class Linear_Regression:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    # KHỞI TẠO HÀM TRIANNING TÍNH HỆ SỐ BẰNG PHƯƠNG TRÌNH CHUẨN
    def fit(self):
        # Thêm cột 1 vào X để tính toán hệ số chặn b
        X = np.hstack((np.ones((self.X.shape[0], 1)), self.X))
        # Tính ma trận chuyển vị của X
        XT = X.T
        # Tính XTX = X^T * X
        XTX = np.dot(XT, X)
        # Tính (XTX)^(-1)
        inv_XTX = np.linalg.inv(XTX)
        # Tính (XTX)^(-1) * X^T
        inv_XTX_XT = np.dot(inv_XTX, XT)
        # Tính kết quả cuối cùng (X^TX)^(-1) * X^T * Y
        beta=np.dot(inv_XTX_XT, self.Y)
        # gán giá trị số chặn và hệ số
        self.intercept = beta[0]
        self.coef = beta[1::]
            
    def predict(self,X_val, Y_val): # thực hiện tính phương trình hồi quy tuyến tính
        Y_pred = np.dot(X_val,self.coef) + self.intercept
        loss = Evaluate.loss(Y_val, Y_pred)
        return Y_pred, loss

In [9]:
df = dataLoader('/kaggle/input/team2-dataset/Advertising.csv','Sales')
df.hold_out()

lin_reg = Linear_Regression(df.train_data.X, df.train_data.Y)
lin_reg.fit()

print(f'Hệ số: {lin_reg.coef}\nSố chặn b: {lin_reg.intercept}\n')

Y_pred, loss = lin_reg.predict(df.val_data.X,df.val_data.Y)
print("cua anh Tuan đẹp trai nhất vũ trụ")
Y_actual = df.val_data.Y.tolist()
data_rs2show = {'Y_acutal': Y_actual, 'Y_predict': Y_pred}
rs2show = pd.DataFrame(data_rs2show)
print(type(rs2show))
print(rs2show)
print(f"\nloss: {loss}")

cai gi :TV           232.1
Radio          8.6
Newspaper      8.7
Sales         13.4
Name: 199, dtype: float64
Hệ số: [ 0.04701402  0.17854709 -0.00224398]
Số chặn b: 3.0454238538923253

cua anh Tuan đẹp trai nhất vũ trụ
<class 'pandas.core.frame.DataFrame'>
    Y_acutal  Y_predict
0       20.7  19.572719
1       10.9   9.502606
2       19.2  18.302962
3       20.1  19.254732
4       10.4   8.903616
5       11.4  10.123379
6       10.3   9.960535
7       13.2  15.617362
8       25.4  23.128633
9       10.9  12.000701
10      10.1   9.707240
11      16.1  18.641037
12      11.6  10.124634
13      16.6  16.463678
14      19.0  18.102647
15      15.6  15.620683
16       3.2   5.296537
17      15.3  15.113518
18      10.1  10.265707
19       7.3  10.082447
20      12.9  12.444795
21      14.4  14.318155
22      13.3  13.355883
23      14.9  15.077122
24      18.0  17.286144
25      11.9  11.167992
26      11.9  14.486983
27       8.0  10.551875
28      12.2  13.652836
29      17.1  17.25670

<H2 style="margin-left:40vw;color:red"> END!</H2>